# logistic回归进行分类

In [5]:
def bmw_finder(mileage, price):
    if price > 25000:
        return 1
    else:
        return 0

In [6]:
from car_data_fifteen import bmws, priuses

all_car_data = list()
for bmw in bmws:
    all_car_data.append((bmw.mileage, bmw.price, 1))
for prius in priuses:
    all_car_data.append((prius.mileage, prius.price, 0))

all_car_data

[(93404.0, 13999.0, 1),
 (110890.0, 13995.0, 1),
 (94133.0, 13982.0, 1),
 (46778.0, 14599.0, 1),
 (53106.0, 22500.0, 1),
 (58761.0, 24998.0, 1),
 (108816.0, 24947.0, 1),
 (81100.0, 13995.0, 1),
 (90000.0, 8400.0, 1),
 (68613.0, 14995.0, 1),
 (94000.0, 11995.0, 1),
 (92500.0, 10995.0, 1),
 (112081.0, 11995.0, 1),
 (105121.0, 11500.0, 1),
 (92000.0, 2013.0, 1),
 (107953.0, 12999.0, 1),
 (56000.0, 18995.0, 1),
 (101191.0, 18900.0, 1),
 (64365.0, 16998.0, 1),
 (66000.0, 10000.0, 1),
 (76675.0, 12500.0, 1),
 (93015.0, 15995.0, 1),
 (80917.0, 14970.0, 1),
 (96000.0, 16795.0, 1),
 (70000.0, 12999.0, 1),
 (107000.0, 11950.0, 1),
 (78000.0, 18995.0, 1),
 (78000.0, 15000.0, 1),
 (92000.0, 2013.0, 1),
 (57624.0, 21963.0, 1),
 (77854.0, 16995.0, 1),
 (48310.0, 22998.0, 1),
 (51656.0, 20998.0, 1),
 (62410.0, 19991.0, 1),
 (39332.0, 29995.0, 1),
 (31420.0, 21000.0, 1),
 (41267.0, 22450.0, 1),
 (73000.0, 19999.0, 1),
 (94608.0, 11995.0, 1),
 (67000.0, 24964.0, 1),
 (50000.0, 18985.0, 1),
 (73601.0, 1

In [7]:
def test_classifier(classifier, data):
    trues = 0
    falses = 0
    for mileage, price, is_bmw in data:
        if classifier(mileage, price) == is_bmw:
            trues += 1
        else:
            falses += 1
    return trues / (trues + falses)

test_classifier(bmw_finder, all_car_data)

0.59

In [9]:
# 更新classifier函数
def test_classifier(classifier, data, verbose=False):
    true_positives = 0
    true_negatives = 0
    false_positives = 0
    false_negatives = 0

    for mileage, price, is_bmw in data:
        predicted = classifier(mileage, price)

        if predicted and is_bmw: # 预测为宝马，且真的是宝马（正确结果）
            true_positives += 1
        elif predicted: # 预测为宝马，但实际不是
            false_positives += 1
        elif is_bmw: # 预测不是宝马，但实际是
            false_negatives += 1
        else: # 预测不是宝马，实际也不是宝马（正确结果）
            true_negatives += 1

    if verbose:
        print("true positives:", true_positives)
        print("true negatives:", true_negatives)
        print("false positives:", false_positives)
        print("false negatives:", false_negatives)

    total = true_positives + true_negatives

    return total / len(data)

test_classifier(bmw_finder, all_car_data, True)

true positives: 18
true negatives: 100
false positives: 0
false negatives: 82


0.59

In [10]:
def bmw_finder2(mileage, price):
    if price > 20000:
        return 1
    else:
        return 0
test_classifier(bmw_finder2, all_car_data, True)

true positives: 48
true negatives: 99
false positives: 1
false negatives: 52


0.735

In [11]:
def decision_boundary_classify(mileage, price):
    if price > 21000 - 0.07 * mileage:
        return 1
    else:
        return 0

test_classifier(decision_boundary_classify, all_car_data)

0.805

In [18]:
def constant_price_classifier(cutoff_price):
    def c(x, p):
        if p > cutoff_price:
            return 1
        else:
            return 0
    return c

def cutoff_accuracy(cutoff_price):
    c = constant_price_classifier(cutoff_price)
    return test_classifier(c, all_car_data)

all_prices = [price for mileage, price, is_bmw in all_car_data]
max(all_prices, key=cutoff_accuracy)

17998.0

In [26]:
# 缩放原始汽车数据

def make_scale(data):
    min_val = min(data)
    max_val = max(data)

    def scale(x):
        return (x-min_val) / (max_val-min_val)

    def unscale(y):
        return y * (max_val - min_val) + min_val

    return scale, unscale

price_scale, price_unscale = make_scale([x[1] for x in all_car_data])
mileage_scale, mileage_unscale = make_scale([x[0] for x in all_car_data])

scaled_data = [(mileage_scale(mileage), price_scale(price), is_bmw) for mileage, price, is_bmw in all_car_data]

In [28]:
# sigmoid函数

from math import *

# 用于拟合的函数，就是模型
def sigmoid(x):
    return 1 / (1 + exp(-x))


def make_logistic(a, b, c):
    def l(x, p):
        return sigmoid((a * x + b * p + c))
    return l

def simple_logistic_cost(a, b, c):
    l = make_logistic(a, b, c)
    errors = [abs(is_bmw - l(x, p)) for x, p, is_bmw in scaled_data]
    return sum(errors)

def point_cost(l, x, p, is_bmw):
    wrong = 1 - is_bmw
    return -log(abs(wrong - l(x, p)))

def logistic_cost(a, b, c):
    l = make_logistic(a, b, c)
    errors = [point_cost(l, x, p, is_bmw) for x, p, is_bmw in scaled_data]
    return sum(errors)

In [32]:
# 梯度下降法
from part1lib import *

def secant_slope(f,xmin,xmax):
    return (f(xmax) - f(xmin)) / (xmax - xmin)

def approx_derivative(f,x,dx=1e-6):
    return secant_slope(f,x-dx,x+dx)

def approx_gradient3(f,x0,y0,z0,dx=1e-6):
    partial_x = approx_derivative(lambda x:f(x,y0,z0),x0,dx=dx)
    partial_y = approx_derivative(lambda y:f(x0,y,z0),y0,dx=dx)
    partial_z = approx_derivative(lambda z:f(x0,y0,z),z0,dx=dx)
    return (partial_x,partial_y,partial_z)
    
def gradient_descent3(f,xstart,ystart,zstart,tolerance=1e-6,max_steps=1000):
    x = xstart
    y = ystart
    z = zstart
    grad = approx_gradient3(f,x,y,z)
    steps = 0
    while length(grad) > tolerance and steps < max_steps:
        x -= 0.01 * grad[0]
        y -= 0.01 * grad[1]
        z -= 0.01 * grad[2]
        grad = approx_gradient3(f,x,y,z)
        steps += 1
    return x,y,z

result = gradient_descent3(logistic_cost, 1, 1, 1, max_steps=100)
print(result)
result = gradient_descent3(logistic_cost, 1, 1, 1, max_steps=1000)
print(result)
result = gradient_descent3(logistic_cost, 1, 1, 1, max_steps=8000)
print(result)

def best_logistic_classifier(x, p):
    l = make_logistic(3.716700347820625, 11.422062473075188, -5.596878401366997)
    if l(x, p) > 0.5:
        return 1
    else:
        return 0

test_classifier(best_logistic_classifier, scaled_data) # 准确率在80%左右

(-0.09359759840122889, 4.668775358379191, -1.894019518985721)
(3.2401157229672175, 10.553176801101781, -5.1272840205583465)
(3.716700347820625, 11.422062473075188, -5.596878401366997)


0.8